In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer('intfloat/e5-small-v2')
input_texts = [
    'query: how much protein should a female eat',
    'query: summit define',
    "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."
]



modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
# embeddings = model.encode(queries + passages)
# scores = util.cos_sim(embeddings[:2], embeddings[2:]) * 100
# print(scores.tolist())
# [[86.71537780761719, 36.645721435546875], [35.00497055053711, 82.07388305664062]]

In [4]:
embeddings = model.encode(input_texts, normalize_embeddings=True)


In [21]:
data = pd.read_csv('data/flowershopdata.csv')
data

,Name,Description,Needs,Season
0,PERIWINKLE,"Perfect as a ground cover, the flowers resembl...",Needs droughty.,Blooms in mid-spring to mid-fall.
1,BELLFLOWER,"Known for their resemblance to bells, these fl...",Needs well-drained soil.,Blooms in early summer to early fall.
2,LILY,A highly recognized flower which has bright co...,Needs well-drained soil.,Blooms in spring to early fall.
3,TULIP,"Great as a cut flower, they come in every colo...",Needs well-drained soil.,Blooms in spring.
4,ROSE,Over 100 species of rose exist along with almo...,Needs well-drained soil.,Blooms in late spring to early fall.
5,ZINNIA,Large blooms in vibrant colors that attract bu...,Needs well-drained soil.,Blooms in mid-summer to mid-fall.
6,BLOODROOT,Named for its red colored sap that flows from ...,Needs well-drained soil.,Blooms in early spring.
7,VIOLET,"A common house plant, they come in many colors...",Needs well-drained soil.,Blooms in early spring to early fall.
8,DAFFODIL,Known for their trumpet-like flowers and brigh...,Needs well-drained soil.,Blooms in early spring.
9,MARIGOLD,Popular for their vibrant orange and yellow bl...,Needs full sun.,Blooms in summer to fall.
